In [1]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import h5py

# ff = h5py.File("raw_chess_data.hdf5", "r")

# whiteWins2 = ff['white']
# blackWins2 = ff['black']
# with h5py.File('raw_chess_data.hdf5','r') as f:
#     whiteWins2 = f['white']
#     blackWins2 = f['black']
#     whiteWins = np.random.permutation(whiteWins2[100000:])[:100000]
#     blackWins = np.random.permutation(blackWins2[100000:])[:100000]

x = np.random.rand(500,50)

C:\Users\neal\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# training an autoencoder:
input_dim = x.shape[1]
a1 = 30
a2 = 10

# first autoencoder layer: input x1, compress to a1 dimensions, expand back to original values 
inputs = Input(shape=(x.shape[1],))
l1 = Dense(a1, activation = 'selu')(inputs)
out = Dense(x.shape[1], activation = 'selu')(l1)
model = Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
          loss='mse', metrics=['mse'])
# need to give x as the inputs and outputs
model.fit(x, x, epochs=70, 
          batch_size = 32,
          verbose=1)

Epoch 1/70
500/500 [==============================] - 0s 866us/step - loss: 0.7621 - mean_squared_error: 0.7621
Epoch 2/70
500/500 [==============================] - 0s 134us/step - loss: 0.3627 - mean_squared_error: 0.3627
Epoch 3/70
500/500 [==============================] - 0s 104us/step - loss: 0.1919 - mean_squared_error: 0.1919
Epoch 4/70
500/500 [==============================] - 0s 97us/step - loss: 0.1533 - mean_squared_error: 0.1533
Epoch 5/70
500/500 [==============================] - 0s 107us/step - loss: 0.1408 - mean_squared_error: 0.1408
Epoch 6/70
500/500 [==============================] - 0s 102us/step - loss: 0.1304 - mean_squared_error: 0.1304
Epoch 7/70
500/500 [==============================] - 0s 91us/step - loss: 0.1219 - mean_squared_error: 0.1219
Epoch 8/70
500/500 [==============================] - 0s 107us/step - loss: 0.1149 - mean_squared_error: 0.1149
Epoch 9/70
500/500 [==============================] - 0s 101us/step - loss: 0.1088 - mean_squared_error: 0

In [6]:
# train the second layer of the autoencoder
# this is where the functional API comes in handy - we can actually just call l1 again and 
# get the already trained weights, but if we don't want to keep
# training it, we can just pull the weights out instead and set trainable = false

# first, we need the outputs:
m2 = Model(inputs=inputs, outputs=l1)
m2.compile(optimizer='adam',
              loss='mse')
x2 = m2.predict(x)

# next, we can build another layer on top of l1, but that will cause l1 to continue to train, so it may not be the best:
l2 = Dense(a2, activation = 'selu')(l1)
out = Dense(a1, activation = 'selu')(l2)
model = Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='mse', metrics=['mse'])
model.fit(x, x2, epochs=70, 
          batch_size = 32,
          verbose=1)

Epoch 1/70
500/500 [==============================] - 0s 891us/step - loss: 0.9934 - mean_squared_error: 0.9934
Epoch 2/70
500/500 [==============================] - 0s 121us/step - loss: 0.5384 - mean_squared_error: 0.5384
Epoch 3/70
500/500 [==============================] - 0s 112us/step - loss: 0.3503 - mean_squared_error: 0.3503
Epoch 4/70
500/500 [==============================] - 0s 131us/step - loss: 0.2444 - mean_squared_error: 0.2444
Epoch 5/70
500/500 [==============================] - 0s 124us/step - loss: 0.1748 - mean_squared_error: 0.1748
Epoch 6/70
500/500 [==============================] - 0s 105us/step - loss: 0.1333 - mean_squared_error: 0.1333
Epoch 7/70
500/500 [==============================] - 0s 115us/step - loss: 0.1128 - mean_squared_error: 0.1128
Epoch 8/70
500/500 [==============================] - 0s 111us/step - loss: 0.1032 - mean_squared_error: 0.1032
Epoch 9/70
500/500 [==============================] - 0s 109us/step - loss: 0.0983 - mean_squared_error:

In [7]:
# alternatively, we can extract the weights from l1 (or from the model) and set them into a new layer that's frozen:
x3 = m2.predict(x)
print(x2[0], x3[0])
# we can see that l1 has been trained even in the new model. Sort of neat, but not necessarily ideal.

[-1.2370961   0.40166467 -0.62386763 -0.74518055 -0.27684033  0.48909643
 -1.2642627   0.5088347  -1.2736311  -1.4862788   0.44073248 -1.2403533
  0.5399281  -0.37873524 -1.4256694  -0.8731435   0.9555403  -1.1208768
  0.39457324  0.15743172 -0.7520019   1.669019    1.8687142  -0.39997151
 -0.20438246  0.06255385  0.34523028 -0.47740695 -0.6081827   0.3524968 ] [-1.1070328  -0.14766712 -1.0270394  -0.8163393  -1.2066611   1.9432664
 -1.3438878   0.8914418  -1.1711365  -1.4491794   0.76663774 -1.3006308
  0.32917118  0.35923654 -1.0142576  -1.3809918   1.3902638  -0.94833666
  1.0981576   1.2987351   0.20346548  1.3478832   1.5440772  -0.67308426
 -1.3389326   1.101044    0.7584531  -0.56174517 -1.2142802   0.7316164 ]


In [10]:
# we can try to set the layers to frozen, though. let's keep it how it is right now, freeze the l1 layer, and repeat the above steps:
for l in m2.layers: 
    l.trainable=False
l2 = Dense(a2, activation = 'selu')(l1)
out = Dense(a1, activation = 'selu')(l2)
model = Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='mse', metrics=['mse'])
model.fit(x, x3, epochs=70, 
          batch_size = 32,
          verbose=0)
x4 = m2.predict(x)
print(x3[0], x4[0])
# sweeeeeeet it works! 
# typically, you don't want to continue training an autoencoder anyway, so this is perfect.

[-1.1070328  -0.14766712 -1.0270394  -0.8163393  -1.2066611   1.9432664
 -1.3438878   0.8914418  -1.1711365  -1.4491794   0.76663774 -1.3006308
  0.32917118  0.35923654 -1.0142576  -1.3809918   1.3902638  -0.94833666
  1.0981576   1.2987351   0.20346548  1.3478832   1.5440772  -0.67308426
 -1.3389326   1.101044    0.7584531  -0.56174517 -1.2142802   0.7316164 ] [-1.1070328  -0.14766712 -1.0270394  -0.8163393  -1.2066611   1.9432664
 -1.3438878   0.8914418  -1.1711365  -1.4491794   0.76663774 -1.3006308
  0.32917118  0.35923654 -1.0142576  -1.3809918   1.3902638  -0.94833666
  1.0981576   1.2987351   0.20346548  1.3478832   1.5440772  -0.67308426
 -1.3389326   1.101044    0.7584531  -0.56174517 -1.2142802   0.7316164 ]


In [13]:
# so, to add another layer, we'd do the same as before:
# we need the outputs again:
m3 = Model(inputs=inputs, outputs=l2)
m3.compile(optimizer='adam',
              loss='mse')
x5 = m3.predict(x)
for l in m3.layers: 
    l.trainable=False

# we can build another layer on top of l2
a3 = 5
l3 = Dense(a3, activation = 'selu')(l2)
out = Dense(a2, activation = 'selu')(l3)
model = Model(inputs=inputs, outputs=out)
model.compile(optimizer='adam',
              loss='mse', metrics=['mse'])
model.fit(x, x5, epochs=70, 
          batch_size = 32,
          verbose=0)

In [15]:
# once that's done, to get the autoencoder, we create a model and freeze the layers:
m4 = Model(inputs=inputs, outputs=l3)
m4.compile(optimizer='adam',
              loss='mse')
for l in m4.layers: 
    l.trainable=False
    
# then we use the functional API to build more network after it:
# if we are using Selu, we should use AlphaDropout ( it's explained in the paper cited in keras documentation, basically when a neuron is off from dropout
# the value it predict should be 0, but using normal dropout the value it gives is -alpha, so they made a special version
# of dropout that corrects for this.)
from keras.layers import AlphaDropout
y = np.random.rand(500,2)
deep1 = Dense(256, activation = 'selu')(l3)
deep2 = Dense(256, activation = 'selu')(deep1)
drop2 = AlphaDropout(0.5)(deep2)
out = Dense(y.shape[1], activation='linear')(drop2)
m5 = Model(inputs=inputs, outputs=out)
m5.compile(optimizer='adam',
              loss='mse')
m5.fit(x, y, epochs=70, 
          batch_size = 32,
          verbose=1)

Epoch 1/70
500/500 [==============================] - 1s 1ms/step - loss: 1.3307
Epoch 2/70
500/500 [==============================] - 0s 106us/step - loss: 0.8427
Epoch 3/70
500/500 [==============================] - 0s 116us/step - loss: 0.5186
Epoch 4/70
500/500 [==============================] - 0s 112us/step - loss: 0.3435
Epoch 5/70
500/500 [==============================] - 0s 107us/step - loss: 0.2508
Epoch 6/70
500/500 [==============================] - 0s 119us/step - loss: 0.1882
Epoch 7/70
500/500 [==============================] - 0s 119us/step - loss: 0.1595
Epoch 8/70
500/500 [==============================] - 0s 106us/step - loss: 0.1352
Epoch 9/70
500/500 [==============================] - 0s 129us/step - loss: 0.1294
Epoch 10/70
500/500 [==============================] - 0s 105us/step - loss: 0.1246
Epoch 11/70
500/500 [==============================] - 0s 114us/step - loss: 0.1101
Epoch 12/70
500/500 [==============================] - 0s 109us/step - loss: 0.1092
Epo

In [17]:
pred = m5.predict(x)
print(pred[0], y[0])

[0.8927445 1.0677538] [0.0176113  0.90055596]
